In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [ ]:
import numpy as np
import pandas as pd

# [Adult Data Set](https://archive.ics.uci.edu/ml/datasets/Adult)

Predict whether income exceeds $50K/yr based on census data.
Also known as "Census Income" dataset.

**Attributes**

- age
- workclass
  - Represents the employment status of an individual
- fnlwgt
  - Final weight which is the number of people the census believes the entry represents
  - People with similar demographic characteristics should have similar weights
    - This only applies within state.
- education
  - The highest level of education achieved by an individual
- education-num
  - The highest level of education achieved in numerical form
- marital-status
- relationship
  - Represents what this individual is relative to others
- race
- sex
- capital-gain
- capital-loss
- hours-per-week
  - The hours an individual has reported to work per week
- native-country

# Data Loading

## Data Files

I have cut the [original data](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data)
into 4 seprate files.

https://github.com/sesise0307/pydata2021-eda/tree/main/data

![Data List](../image/data_list.png)

## Loading a file

In [ ]:
if IN_COLAB:
    data_file = 'https://raw.githubusercontent.com/sesise0307/pydata2021-eda/main/data/adult-0.data'
else:
    data_file = '../data/adult-0.data'

df_0 = pd.read_csv(data_file)
    
df_0.shape

In [ ]:
df_0.head()

In [ ]:
names = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education_num',
    'marital_status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital_gain',
    'capital_loss',
    'hours_per_week',
    'native_country',
    'income',
]

In [ ]:
df_0 = pd.read_csv(
    data_file,
    names=names,
    skipinitialspace=True,  # Skip spaces after delimiter
)

df_0.shape

In [ ]:
df_0.head()

## Merging many files

In [ ]:
df_list = list()

for i in range(4):
    print(f'Reading a csv file: adult-{i}.data')
    
    if IN_COLAB:
        data_file = f'https://raw.githubusercontent.com/sesise0307/pydata2021-eda/main/data/adult-{i}.data'
    else:
        data_file = f'../data/adult-{i}.data'
    
    df_list.append(        
        pd.read_csv(
            data_file,
            names=names,
            skipinitialspace=True,
        )
    )

df = pd.concat(df_list, ignore_index=True)

df.shape

In [ ]:
df.head(10)

# Essential Check

## info() and describe()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(exclude=np.number)

## Unique Values

In [ ]:
print('Unique workclass: ', df['workclass'].unique())
print('The number of unique workclass: ', df['workclass'].nunique())

In [ ]:
for column in df.columns:
    n_unique = df[column].nunique()
    
    if df[column].nunique() < 50:
        print(f'{column} ({n_unique}): {df[column].unique()}\n')
    else:
        print(f'{column} ({n_unique})\n')

> Note that if a variable contains only 1 unique value,
> it means that the variable does not have any information.

## Missing Values

### Visualization with missingno

In [ ]:
import missingno as msno

msno.matrix(df);

In [ ]:
msno.bar(df);

In [ ]:
# Replace '?' to NaN
df = df.replace('?', np.nan)

In [ ]:
msno.matrix(df);

In [ ]:
msno.bar(df);

### Manual computation of NaN ratio

In [ ]:
df.isna().head()

In [ ]:
df.isna().sum()

In [ ]:
df.isna().sum() / df.shape[0]

In [ ]:
(
    (df.isna().sum() / df.shape[0])
    .sort_values(ascending=False)  # Sort by NaN Ratio
)

### Handling Missing Values

![Handling Missing Data](../image/handle_missing_data.png)

Source: https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4

#### fillna()

##### workclass fillna() with mode

In [ ]:
workclass_mode = df['workclass'].mode()[0]
workclass_mode

In [ ]:
df['workclass'].unique()

In [ ]:
df['workclass'] = df['workclass'].fillna(workclass_mode)

In [ ]:
df['workclass'].unique()

##### occupation fillna with "Unknown"

In [ ]:
df['occupation'].unique()

In [ ]:
df['occupation'] = df['occupation'].fillna('Unknown')

In [ ]:
df['occupation'].unique()

#### dropna()

In [ ]:
# Drop rows with at least one NaN values
df = df.dropna().reset_index(drop=True)

#### Verification

In [ ]:
msno.bar(df);

## [Sidetable](https://github.com/chris1610/sidetable)

Builds simple but useful summary tables of your pandas DataFrame.

Usage is straightforward. Install and import sidetable. Then access it through the new .stb accessor on your DataFrame.

In [ ]:
if IN_COLAB:
    !pip install sidetable

In [ ]:
import sidetable

In [ ]:
df.stb.freq(['race', 'sex'])

In [ ]:
df.stb.counts()

In [ ]:
df.stb.missing(style=True)

In [ ]:
(
    df
    .groupby(['race', 'sex'])
    [['capital_gain']]
    .mean()
)

In [ ]:
(
    df
    .groupby(['race', 'sex'])
    [['capital_gain']]
    .mean()
    .stb
    .subtotal()
)

# (Primitive) Preprocessing & Feature Engineering

## Age Grouping

In [ ]:
df['age'].describe()

In [ ]:
df['age_group'] = pd.cut(
    df['age'],
    bins=range(10, 101, 10),
    right=False,
    labels=[f'{age_start}~{age_start + 9}'
            for age_start in range(10, 100, 10)]
)

In [ ]:
df['age_group'].cat.categories

In [ ]:
df[['age', 'age_group']].tail(10)

## Cateogry and Category Ordering

In [ ]:
(
    df
    .groupby('education')['education_num']
    .unique()
    .sort_values()
)

In [ ]:
education_order = (
    df
    .groupby('education')['education_num']
    .unique()
    .sort_values()
    .index
)

In [ ]:
df['education'] = df['education'].astype(
    pd.CategoricalDtype(categories=education_order,
                        ordered=True)
)

In [ ]:
df['education'].head()

In [ ]:
df['education_num'] = df['education_num'].astype(
    pd.CategoricalDtype(ordered=True)
)

In [ ]:
df['education_num'].head()

## Captial gain and loss

In [ ]:
df[['capital_gain', 'capital_loss']].tail(10)

In [ ]:
(  # Check if capital gain and capital loss appear at the same time
    df[['capital_gain', 'capital_loss']]
    .astype(bool)
#     .sum(axis='columns')
#     .max()
)

In [ ]:
df['capital_gain'] = df['capital_gain'] - df['capital_loss']
df = df.drop(columns='capital_loss')

In [ ]:
df['capital_gain'].describe()

## Income exceeds $50K/yr

In [ ]:
df['income_exceed_50k'] = df['income'] == '>50K'

In [ ]:
df[['income', 'income_exceed_50k']].head(10)

## Adding Fake Income (for fun)

In [ ]:
df.head()

In [ ]:
import random

def get_fake_income(row):
    exceed_50k = row['income_exceed_50k']
        
    weight_age = 1.2 if 40 <= row['age'] <= 55 else 1.0
    weight_hours_per_week = 1.2 if 38 <= row['hours_per_week'] else 0.8
    weight_race = 1.2 if row['race'] == 'White' else 0.9
    weight_sex = 1.2 if row['sex'] == 'Male' else 0.9
    
    mu = 35000 * weight_age * weight_hours_per_week * weight_race * weight_sex
    sigma = mu * random.uniform(0.5, 1.5)
    
    while True:  # Guarantee fake income is conform to exceed_50k
        income = int(random.gauss(mu, sigma))
        
        if exceed_50k and 50000 < income:
            return income
        
        elif not exceed_50k and 1000 < income <= 50000:
            return income

In [ ]:
random.seed(42)
df['fake_income'] = df.apply(get_fake_income, axis='columns')

# Save the Preprocessed DataFrame

`pandas` supports various file types for both reading and saving your DataFrame.

![Pandas I/O](../image/pandas_io.png)

Source: https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html

> It seems that `feather` format is an ideal candidate to store the data between Jupyter sessions.
> It shows high I/O speed, doesn’t take too much memory on the disk and doesn’t need any unpacking when loaded back into RAM.
>
> Source: [The Best Format to Save Pandas Data](https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d).

In [ ]:
if IN_COLAB:
    df.to_feather('adult_preprocessed.feather')
else:
    df.to_feather('../data/preprocessed/adult.feather')

In [ ]:
if IN_COLAB:
    df_feather = pd.read_feather('adult_preprocessed.feather')
else:
    df_feather = pd.read_feather('../data/preprocessed/adult.feather')

df_feather.head()

In [ ]:
df_feather.info()